In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
data =pd.read_csv("C:/Users/HP/workspace/Real Estate Prediction Project/dataset/Bengaluru_House_Data.csv")
data.head()

In [ ]:
data.groupby("area_type")['area_type'].agg('count')

In [ ]:
data['area_type'].value_counts()

In [ ]:
#droping columns that are essentially not useful to determining our house price

data1 = data.drop(['area_type', 'society', 'balcony', 'availability'], axis= 1)

In [ ]:
#handling the na values
data2 = data1.dropna()
data2.shape

In [ ]:
#exploring the size column and clearing and tokenising it
view = data2['size'].unique()

#creating a new tokenized size column
data2['bhk'] = data2['size'].apply(lambda x: int(x.split(" ")[0]))

In [ ]:
#exploring the total_sqft column
vi = data2.total_sqft.unique()
# defining a function to convert the range values in the total_square column

def is_float(x):
    try:
        float(x)
    except:
        return False
    return True
        

In [ ]:
data2[~data2["total_sqft"].apply(is_float)].head(10)

In [ ]:
#writing a function  to convert the range values to float
def convert_sqft_to_num(x):
    token = x.split("-")
    if len(token) == 2:
        return (float(token[0])+float(token[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
data3 = data2.copy()
data3['total_sqft'] = data3['total_sqft'].apply(convert_sqft_to_num)


In [ ]:
#feature engineering and dimensionality reduction 
#creating price per square fit column

data4 = data3.copy()
data4['price_per_sqft'] = data4['price']*100000/data4['total_sqft']

In [ ]:
#exploring the location column
data4.location == data4.location.apply(lambda x: x.strip())
location_stats = data4['location'].value_counts(ascending=False)

location_stats1 = data4.groupby('location')['location'].agg('count').sort_values(ascending=False)

In [ ]:
len(location_stats[location_stats<=10])

In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
#converting location that are less that 10 to other
data4.location = data4.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(data4.location.unique())

In [ ]:
#outlier detection and remover
#checcking the data points where the sqft is less than 300(threshold)
check = data4[data4.total_sqft/data4.bhk<300]

#removing the outliers
data5 = data4[~(data4.total_sqft/data4.bhk<300)]

In [ ]:
data5.price_per_sqft.describe()

In [ ]:
#writing a function that can remove extreme cases of high price_per_sqft per location
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        sd = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-sd)) & (subdf.price_per_sqft<=(m+sd))]
        df_out = pd.concat([df_out, reduced_df], ignore_index=True)
    return df_out
    
data6 = remove_pps_outliers(data5)
data6.shape

In [ ]:
#function for data visualisation of the 2&3 bedroom case senerior

def plot_scatter_chart(df, location):
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    plt.rcParams['figure.figsize'] = (15, 10)
    plt.scatter(bhk2.total_sqft, bhk2.price, color='blue', label = '2 BHK', s=50)
    plt.scatter(bhk3.total_sqft, bhk3.price, marker='+', color='green', label = '3 BHK', s=50)
    plt.xlabel("Total square Feet Area")
    plt.ylabel("Price Per Square Feet")
    plt.title(location)
    plt.legend()


plot_scatter_chart(data6, "Rajaji Nagar")

In [ ]:
#fnction to remove those 2 BHK apartment whose price_per_sqft is less than
#mean price price_per_sqft of 1bHK apartment

def remove_bhk_outlier(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft), 
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices, axis = 'index')
            
data7 = remove_bhk_outlier(data6)
data7.shape
            

In [ ]:
plot_scatter_chart(data7, "Hebbal")

In [ ]:
#ploting  a histogram to see hoow many apartment are there per_sqft Area
plt.rcParams['figure.figsize'] = (20,10)
plt.hist(data7.price_per_sqft, rwidth=0.8)
plt.xlabel('Price Per Square Feet')
plt.ylabel('count')

In [ ]:
#removing the outliers on the bath columns. every datapoint having 2bathroom more than bedbroom is categorized 
# as an outlier

data8 = data7[data7.bath < data7.bhk+2]
data8.shape

In [ ]:
#droping the unneccessary features like price_per_sqft and size

data9 = data8.drop(['size', 'price_per_sqft'], axis = 'columns')

In [ ]:
#creating a machine learning model for price prediction 
#using onehot encoding on the location column

dummies = pd.get_dummies(data9.location)

#concatingn the dummies + the original data
data10 = pd.concat([data9, dummies.drop('other', axis = 'columns')], axis = 'columns')

data11 = data10.drop('location', axis= 'columns')
data11.head(10)

Building the model


In [ ]:
#creating x and y datapoint of training the machine learning model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


X = data11.drop('price', axis ='columns')
y = data11.price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_model.score(X_test, y_test)

In [ ]:
#using kfold validation
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size = 0.2, random_state = 0)
cross_val_score(lr_model, X, y, cv=cv )

In [ ]:
#using gridsearchcv
#hyper-parameter tunning
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso

def find_best_model_using_gridsearchcv(x,y):
    algos = {
        'linear_regression': {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
            
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
        }
    }
    
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score = False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
        
    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

find_best_model_using_gridsearchcv(X,y)

In [ ]:
#writing the predict function

def predict_price(location, sqft, bath, bhk):
    loc_index = np.where(X.columns==location)[0]
    
    x=np.zeros(len(X.columns))
    x[0]=sqft
    x[1]=bath
    x[2]=bhk
    if loc_index >= 0:
        x[loc_index] =  1
    return lr_model.predict([x])[0]
        


In [ ]:
predict_price('1st phase JP Nagar', 1000, 3,3)

In [ ]:
predict_price('Indira Nagar', 1000, 2, 2)

In [ ]:
#exporting the model to the pickle file
import pickle
with open ('banglore_home_price_model.pickle', 'wb') as f:
    pickle.dump(lr_model, f)

In [ ]:
#exporting the columns labels which will be usefull for prediction

import json
columns = {
    'data_columns': [col.lower() for col in X.columns]
}
with open("columns.json", "w") as f:
    f.write(json.dumps(columns))